## Dimensionality Reduction Assignment

- Generate a data set of 900 3-dimensional data vectors, which stem from two classes—the first 100 vectors from a zero-mean Gaussian distribution with covariance matrix:
    - S1 = [0.5 0 0; 0 0.5 0; 0 0 0.01]. 
- The rest are grouped into 8 groups of 100 vectors.
- Each group stems from a Gaussian distribution.
- All of these distributions share the covariance matrix: 
    - S2 = [1 0 0; 0 1 0; 0 0 0.01], while their means are:

    ![means](images/image1.png "means")

In [10]:
## sklearn.datasets.make_blobs: Generate isotropic Gaussian blobs for clustering.
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html

from sklearn.datasets import make_blobs
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA

In [2]:
# S1 = [[0.5, 0, 0], [0, 0.5, 0], [0, 0, 0.01]]
# S2 = [[1, 0, 0], [0, 1, 0], [0, 0, 0.01]]

# smarter :)
S1 = np.diag([0.5, 0.5, 0.01])
S2 = np.diag([1, 1, 0.01]) 

In [3]:
# Numpy module have a lot of useful functions for generating random numbers, including Gaussian distributions.
# https://numpy.org/doc/stable/reference/random/index.html

rng = np.random.default_rng(42)
a = 20.0

mean_c1 = np.array([0.0, 0.0, 0.0])
means_c2 = np.array([
    [ a,    0,   0],
    [ a/2,  a/2, 0],
    [ 0,    a,   0],
    [-a/2,  a/2, 0],
    [-a,    0,   0],
    [-a/2, -a/2, 0],
    [ 0,   -a,   0],
    [ a/2, -a/2, 0]
], dtype=float)

In [7]:
# 100 samples from a zero-mean Gaussian distribution with covariance matrix S1
X1 = rng.multivariate_normal(mean_c1, S1, 100)

# 800 samples from 8 groups of 100 vectors, each group from a Gaussian distribution with covariance matrix S2 and means from means_c2
groups = []
for mean in means_c2:
    group = rng.multivariate_normal(mean, S2, 100)
    groups.append(group)
X2 = np.vstack(groups)

# Final set
X = np.vstack((X1, X2))
y = np.array([0] * 100 + [1] * 800)

print(f"X.shape: {X.shape}, y.shape: {y.shape}")

X.shape: (900, 3), y.shape: (900,)


In [11]:
# to pandas DataFrame to use plotly express
df = pd.DataFrame(X, columns=['X1', 'X2', 'X3'])
df['y'] = y

In [27]:
# a) Plot the 3-dimensional data set and view it from different angles to get a feeling of how the data is spread in the 3-dimensional space

fig = px.scatter_3d(
    df,
    x='X1', y='X2', z='X3',
    color='y',
    title='3D Distribution'
)

fig.show()

In [ ]:
# b) Perform LDA on the previous dataset. Project the data on the subspace spanned by the eigenvectors that correspond to the nonzero eigenvalues of the matrix product S−1w Sb. Comment on the results.
lda = LDA(n_components=1)
Z = lda.fit_transform(X, y).ravel()   # 1D projection


In [29]:
df_proj = pd.DataFrame({
    "LDA_1D": Z,
    "Class": np.where(y==0, "Class 1", "Class 2")
})
fig = px.histogram(
    df_proj, x="LDA_1D", color="Class",
    barmode="overlay", nbins=60,
    title="LDA - 1D Projection",
)
fig.show()


- Generate a 3-dimensional Archimedes spiral as a pack of 11 identical 2-dimensional Archimedes spirals, one above the other. 
- A 2-dimensional spiral is described in polar coordinates by the equation r = aθ, where a is a user-defined parameter. 
- In our case, the points of a 3-dimensional spiral are generated as follows: For θ, take the values from θinit to θfin with step θstep and compute
    - r = a θ
    - x = r cosθ
    - y = r sinθ

- The 11 points of the form (x, y, z), where z = −1, −0.8, −0.6, ..., 0.8, 1, are points of the spiral. Use a = 0.1, θinit = 0.5, θfin = 2.05 ∗ π, θstep = 0.2. 
- Plot the 3-dimensional spiral so that all points of the same 2-dimensional spiral are plotted using the same symbol and all groups of 11 points of the form (x, y,z), where x and y are fixed and z takes the values −1, −0.8, −0.6, ..., 0.8, 1, are plotted using the same color.
- Perform linear PCA on the dataset, using the first two principal components. Plot the results.
- Apply kernel PCA for a dimension m=2 for different Gaussian kernel parameters and plot the results.


In [2]:
# params
a = 0.1
theta_init = 0.5
theta_fin  = 2.05 * np.pi
theta_step = 0.2
z_levels   = np.linspace(-1, 1, 11)  # -1, -0.8, ..., 0.8, 1

# angles
thetas = np.arange(theta_init, theta_fin + 1e-9, theta_step)

# cartesian coordinates
r = a * thetas
x = r * np.cos(thetas)
y = r * np.sin(thetas)

In [3]:
# stack the same spiral at 11 z heights
records = []
for iz, z in enumerate(z_levels):
    for it, (xi, yi) in enumerate(zip(x, y)):
        # id_theta identifies the "vertical column" (same x,y; 11 z's)
        # id_z identifies which spiral (z plane)
        records.append((xi, yi, z, it, iz))

df = pd.DataFrame(records, columns=["x","y","z","id_theta","id_z"])

In [15]:
fig3d = px.scatter_3d(
    df, x="x", y="y", z="z",
    color="id_theta", symbol="id_z",
    title="3d spiral (colors = columns θ, symbols = z axis)",
    opacity=0.8
)
fig3d.update_traces(marker=dict(size=4))
fig3d.show()


In [6]:
# data matrix
X = df[["x","y","z"]].values

# PCA is sensitive to the scale of the data
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [7]:
# PCA 2D
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_std)

df_pca = df.copy()
df_pca["PC1"] = X_pca[:,0]
df_pca["PC2"] = X_pca[:,1]

print("Variance (PC1, PC2):", pca.explained_variance_ratio_)

Variance (PC1, PC2): [0.40866074 0.33333333]


In [16]:
fig_pca = px.scatter(
    df_pca, x="PC1", y="PC2",
    color="id_theta", symbol="id_z",
    title="Linar PCA (2D)",
    opacity=0.85
)
fig_pca.update_traces(marker=dict(size=5))
fig_pca.show()

In [11]:
gammas = [0.1, 1, 2, 5, 10, 20, 30, 40, 50]

In [13]:
# kernel rbf
for gamma in gammas:
    kpca_rbf = KernelPCA(n_components=2, kernel="rbf", gamma=gamma, random_state=42)
    Z_rbf = kpca_rbf.fit_transform(X_std)
    df_rbf = df.copy()
    df_rbf["KPC1"] = Z_rbf[:,0]
    df_rbf["KPC2"] = Z_rbf[:,1]


    fig = px.scatter(
        df_rbf, x="KPC1", y="KPC2",
        color="id_theta", symbol="id_z",
        title=f"Kernel PCA (RBF) — gamma={gamma}",
        opacity=0.85
    )
    fig.update_traces(marker=dict(size=5))
    fig.show()


In [14]:
# kernel polinomial
degrees = [2, 3]
for deg in degrees:
    for gamma in gammas:
        kpca_poly = KernelPCA(
            n_components=2, kernel="poly",
            gamma=gamma, degree=deg, coef0=1.0, random_state=42
        )
        Z_poly = kpca_poly.fit_transform(X_std)
        df_poly = df.copy()
        df_poly["KPC1"] = Z_poly[:,0]
        df_poly["KPC2"] = Z_poly[:,1]
        fig = px.scatter(
            df_poly, x="KPC1", y="KPC2",
            color="id_theta", symbol="id_z",
            title=f"Kernel PCA (Poly) — degree={deg}, gamma={gamma}, coef0=1.0",
            opacity=0.85
        )
        fig.update_traces(marker=dict(size=5))
        fig.show()
